# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
data=pd.read_csv('WeatherPy/city_data.csv')
locations=data[['Lat','Long']]
Humidity=data['Humidity']
data.head()

,City Name,Lat,Long,Max Temp,Cloudiness,Humidity,Wind Speed,Country
0,rikitea,-23.12,-134.97,73.33,100.0,84.0,20.69,PF
1,ancud,-41.87,-73.82,48.99,92.0,97.0,20.00,CL
2,qaanaaq,77.48,-69.36,25.86,98.0,77.0,9.10,GL
3,hobart,-42.88,147.33,48.20,99.0,66.0,12.75,AU
4,mataura,-46.19,168.86,42.01,54.0,87.0,1.41,NZ


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
lat=data['Lat'][1]


In [37]:
hotel_df=pd.DataFrame({'Hotel Name':[],
                       'Hotel Lat':[],
                       'Hotel Lng':[]})

for x in range(len(data)):
    lat=data['Lat'][x]
    lng=data['Long'][x]
    target_coordinates=f'{lat},{lng}'
    target_radius=5000
    target_type='lodging'

    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    response = requests.get(base_url, params=params)

    hotel_data = response.json()
    #print(json.dumps(hotel_data, indent=4, sort_keys=True))

    hotel_df.at[x,'Hotel Name']=hotel_data['results'][0]['name']
    hotel_df.at[x,'Hotel Lat']=hotel_data['results'][0]['geometry']['location']['lat']
    hotel_df.at[x,'Hotel Lng']=hotel_data['results'][0]['geometry']['location']['lng']

hotel_df

,Hotel Name,Hotel Lat,Hotel Lng
0,Pension Maro'i,-23.120620,-134.968753
1,Hotel Arena Gruesa,-41.860358,-73.822419
2,Qaanaaq Hotel,77.469287,-69.231591
3,St Ives Hobart Accommodation Tasmania,-42.891150,147.329667
4,Ellie's Villa,-46.201943,168.855043
5,Bora-Bora Pearl Beach Resort & Spa,-16.475941,-151.772479
6,Albatros Hotel,-54.806967,-68.304891
7,Apartments Mezhdurechensk,53.688791,88.078306
8,Gostinitsa,68.756827,161.327610
9,Hotel JM Puerto del Rosario,28.499081,-13.857706


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [41]:
# Add marker layer ontop of heat map
fig = gmaps.figure(layout=figure_layout)
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
fig.add_layer(heat_layer)
markers_coord=hotel_df[['Hotel Lat', 'Hotel Lng']]
markers=gmaps.marker_layer(markers_coord)
fig.add_layer(markers)
fig
# Display figure


Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…